# IDEA:
Young adults are more likely to choose biking for commuting.

# NULL HYPOTHESIS:

The ratio of subscribers aged 25-34 biking on weekends over biking on weekdays is the same or higher than the ratio of subscribers aged 35-44 biking over weekends to biking on weekdays

**I will use a significance level of alpha = 0.05**

H0 : Old.weekend/Old.week <= Young.weekend/Young.week

H1 : Old.weekend/Old.week > Young.weekend/Young.week

In [29]:
from __future__  import print_function, division
import pylab as pl
import pandas as pd
import numpy as np
import os
import urllib2
import zipfile

%pylab inline

if os.getenv ('PUI2016') is None:
    print ("Must set env variable PUI2016")
if os.getenv ('PUIDATA') is None:
    print ("Must set env variable PUIDATA")

import json
import csv
import requests
import datetime as dt

Populating the interactive namespace from numpy and matplotlib


In [2]:
#developed using answer by David Robinson here http://stackoverflow.com/questions/8814813/saving-a-downloaded-zip-file-w-python
datestring = '201606'
download = "https://s3.amazonaws.com/tripdata/" + datestring + "-citibike-tripdata.zip"
if os.path.isfile(datestring + "-citibike-tripdata.zip"):
    print("zip file already exists")
else:
    request = urllib2.urlopen (download)
    output = open(datestring + "-citibike-tripdata.zip", "w")
    output.write(request.read())
    output.close()

zip file already exists


In [3]:
if os.path.isfile(datestring + "-citibike-tripdata.csv"):
    print ("csv file already exists")
else:
    os.system("unzip " + datestring + "-citibike-tripdata.zip")

csv file already exists


In [30]:
df = pd.read_csv(datestring + "-citibike-tripdata.csv")
df['datetime'] = pd.to_datetime(df['starttime'])
df.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,datetime
0,1470,6/1/2016 00:00:18,6/1/2016 00:24:48,380,W 4 St & 7 Ave S,40.734011,-74.002939,3236,W 42 St & Dyer Ave,40.758985,-73.993800,19859,Subscriber,1972.0,1,2016-06-01 00:00:18
1,229,6/1/2016 00:00:20,6/1/2016 00:04:09,3092,Berry St & N 8 St,40.719009,-73.958525,3103,N 11 St & Wythe Ave,40.721533,-73.957824,16233,Subscriber,1967.0,1,2016-06-01 00:00:20
2,344,6/1/2016 00:00:21,6/1/2016 00:06:06,449,W 52 St & 9 Ave,40.764618,-73.987895,469,Broadway & W 53 St,40.763441,-73.982681,22397,Subscriber,1989.0,1,2016-06-01 00:00:21
3,1120,6/1/2016 00:00:28,6/1/2016 00:19:09,522,E 51 St & Lexington Ave,40.757148,-73.972078,401,Allen St & Rivington St,40.720196,-73.989978,16231,Subscriber,1991.0,1,2016-06-01 00:00:28
4,229,6/1/2016 00:00:53,6/1/2016 00:04:42,335,Washington Pl & Broadway,40.729039,-73.994046,285,Broadway & E 14 St,40.734546,-73.990741,15400,Subscriber,1989.0,1,2016-06-01 00:00:53


In [31]:
df.columns

Index([u'tripduration', u'starttime', u'stoptime', u'start station id',
       u'start station name', u'start station latitude',
       u'start station longitude', u'end station id', u'end station name',
       u'end station latitude', u'end station longitude', u'bikeid',
       u'usertype', u'birth year', u'gender', u'datetime'],
      dtype='object')

In [32]:
df.drop(['tripduration', 'starttime', 'stoptime', 'start station id',
       'start station name', 'start station latitude',
       'start station longitude', 'end station id', 'end station name',
       'end station latitude', 'end station longitude', 'bikeid', 'gender'], axis=1, inplace=True)

In [33]:
df.head()

,usertype,birth year,datetime
0,Subscriber,1972.0,2016-06-01 00:00:18
1,Subscriber,1967.0,2016-06-01 00:00:20
2,Subscriber,1989.0,2016-06-01 00:00:21
3,Subscriber,1991.0,2016-06-01 00:00:28
4,Subscriber,1989.0,2016-06-01 00:00:53


In [118]:
df[df['usertype'].str.contains("Subscriber")]
df.head()

,usertype,birth year,datetime
0,Subscriber,1972.0,2016-06-01 00:00:18
1,Subscriber,1967.0,2016-06-01 00:00:20
2,Subscriber,1989.0,2016-06-01 00:00:21
3,Subscriber,1991.0,2016-06-01 00:00:28
4,Subscriber,1989.0,2016-06-01 00:00:53


In [119]:
df1 = df[np.isfinite(df['birth year'])]
df1.head()

,usertype,birth year,datetime
0,Subscriber,1972.0,2016-06-01 00:00:18
1,Subscriber,1967.0,2016-06-01 00:00:20
2,Subscriber,1989.0,2016-06-01 00:00:21
3,Subscriber,1991.0,2016-06-01 00:00:28
4,Subscriber,1989.0,2016-06-01 00:00:53


In [120]:
df1['age'] = 2016 - df1['birth year']
df1.head()

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,usertype,birth year,datetime,age
0,Subscriber,1972.0,2016-06-01 00:00:18,44.0
1,Subscriber,1967.0,2016-06-01 00:00:20,49.0
2,Subscriber,1989.0,2016-06-01 00:00:21,27.0
3,Subscriber,1991.0,2016-06-01 00:00:28,25.0
4,Subscriber,1989.0,2016-06-01 00:00:53,27.0


In [121]:
df2 = df1.drop(df1[df1.age >= 44].index)
df3 = df2.drop(df2[df2.age <= 25].index)
print(df3.head())
print(len(df3))

     usertype  birth year            datetime   age
2  Subscriber      1989.0 2016-06-01 00:00:21  27.0
4  Subscriber      1989.0 2016-06-01 00:00:53  27.0
5  Subscriber      1974.0 2016-06-01 00:01:01  42.0
6  Subscriber      1986.0 2016-06-01 00:01:05  30.0
7  Subscriber      1986.0 2016-06-01 00:01:24  30.0
748420


In [122]:
df3['weekday'] = df3['datetime'].apply(lambda x: x.weekday())
df3_weekdays = df3[df3['weekday'] <= 5]

In [123]:
len(df3_weekdays)

679213

In [124]:
temp = pd.DatetimeIndex(df3_weekdays['datetime'])
df3_weekdays['date'] = temp.date
df3_weekdays['time'] = temp.time
del df3_weekdays['datetime']
df3_weekdays.head()

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


,usertype,birth year,age,weekday,date,time
2,Subscriber,1989.0,27.0,2,2016-06-01,00:00:21
4,Subscriber,1989.0,27.0,2,2016-06-01,00:00:53
5,Subscriber,1974.0,42.0,2,2016-06-01,00:01:01
6,Subscriber,1986.0,30.0,2,2016-06-01,00:01:05
7,Subscriber,1986.0,30.0,2,2016-06-01,00:01:24


In [125]:
# create a dataframe for trips during commuting hours of 7am-10am and 4pm-7pm
com1 = df3_weekdays[(df3_weekdays['time'] >= dt.time(hour=7)) & (df3_weekdays['time'] <= dt.time(hour=10))]
com2 = df3_weekdays[(df3_weekdays['time'] >= dt.time(hour=16)) & (df3_weekdays['time'] <= dt.time(hour=19))]

com_hours = com1.append(com2)
com_hours.head()

,usertype,birth year,age,weekday,date,time
2486,Subscriber,1973.0,43.0,2,2016-06-01,07:00:00
2487,Subscriber,1982.0,34.0,2,2016-06-01,07:00:00
2491,Subscriber,1983.0,33.0,2,2016-06-01,07:00:10
2492,Subscriber,1985.0,31.0,2,2016-06-01,07:00:16
2494,Subscriber,1979.0,37.0,2,2016-06-01,07:00:22


In [127]:
young = df3_weekdays[df3_weekdays['age'] <= 34]
old = df3_weekdays[df3_weekdays['age'] > 34]

n_young = len(young)
n_old = len(old)
n_total = n_young + n_old

print(n_young, n_old, n_total)

412494 266719 679213


In [132]:
# split commuting trips by age group

old_com_hours = com_hours[com_hours['age'] > 34]
young_com_hours = com_hours[com_hours['age'] <= 34]

print(len(young_com_hours), len(old_com_hours))

199480 135272


In [129]:
per_young = len(young_com_hours) / n_young
print(per_young, n_young)

0.483594912896 412494


In [130]:
per_old = len(old_com_hours) / n_old
print(per_old, n_old)

0.507170467796 266719


In [133]:
# defining the functions that calculate the statistics
# the lambda style definition is NOT APPROVED BY PEP8 and you should avoind it
# but I am putting it here since you may encounter it in your coding life.
# confront the equation in the markdown cell above with these function to understand as they work

p = lambda p0, p1, n0, n1: (p0 * n0 + p1 * n1) / (n0 + n1)
se = lambda p, n0, n1: np.sqrt(p * (1 - p) * (1.0 / n0 + 1.0 / n1)) #standard error
zscore = lambda p0, p1, s : (p0 - p1) / s

# calculations

sp_stdev = se(p(per_young, per_old, n_young, n_old), n_young, n_old)
# print (sp_stdev_mw)
z = zscore(per_young, per_old, sp_stdev)
print ("The z statistics is %.2f"%z) 

The z statistics is -18.98
